In [ ]:
!pip install transformers==2.11.0
!pip install nlp==0.2.0

In [1]:
import numpy as np
import torch
import torch.nn as nn
import transformers
import nlp
from transformers import AutoTokenizer
import dataclasses
from torch.utils.data.dataloader import DataLoader
from transformers.training_args import is_tpu_available
from transformers.trainer import get_tpu_sampler
from transformers.data.data_collator import DataCollator, InputDataClass
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from typing import List, Union, Dict
from time import time
# import logging
# logging.basicConfig(level=logging.INFO)

In [2]:
dataset_dict = {
    "cola": nlp.load_dataset('glue', name="cola"),
    "stsb": nlp.load_dataset('glue', name="stsb"),
#     "qnli": nlp.load_dataset('glue', name="qnli"),
    "wnli": nlp.load_dataset('glue', name="wnli")
    #"mnli": nlp.load_dataset('glue', name="mnli")    
}

In [ ]:
for task_name, dataset in dataset_dict.items():
    print(task_name)
    print(dataset_dict[task_name]["train"][0])
    print()

In [3]:
class MultitaskModel(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        """
        Setting MultitaskModel up as a PretrainedModel allows us
        to take better advantage of Trainer features
        """
        super().__init__(transformers.PretrainedConfig())

        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def create(cls, model_name, model_type_dict, model_config_dict=None):
        """
        This creates a MultitaskModel using the model class and config objects
        from single-task models. 

        We do this by creating each single-task model, and having them share
        the same encoder transformer.
        """
        shared_encoder = None
        taskmodels_dict = {}
        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained(
                model_name, 
                config=model_config_dict[task_name],
            )
            if shared_encoder is None:
                print('*****************')
                #print(cls.get_encoder_attr_name(model))
                
                shared_encoder = getattr(model, model.base_model_prefix)
                #shared_encoder
                print(shared_encoder)
                print('*****************')
            else:
                setattr(model, model.base_model_prefix
                        , shared_encoder)
            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)


    def forward(self, task_name, **kwargs):
        return self.taskmodels_dict[task_name](**kwargs)

In [4]:
model_name = 'bert-base-cased'
multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        "stsb": transformers.AutoModelForSequenceClassification,
        "cola": transformers.AutoModelForSequenceClassification,
        "wnli": transformers.AutoModelForSequenceClassification,
        #"mnli": transformers.AutoModelForSequenceClassification,
    },
    model_config_dict={
        "stsb": transformers.AutoConfig.from_pretrained(model_name, num_labels=1),
        "cola": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        "wnli": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        #"mnli": transformers.AutoConfig.from_pretrained(model_name, num_labels=3)
    },
)

*****************
BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace

In [5]:
max_length = 340

# def convert_to_sst2_features(example_batch):
#     inputs = list(zip(example_batch['sentence']))
#     features = tokenizer.batch_encode_plus(
#         inputs,
#         max_length=max_length,
#         pad_to_max_length=True
#     )
    
#     features['labels'] = example_batch['label']
#     return features

def convert_to_cola_features(example_batch):
    #print(example_batch)
    inputs = example_batch['sentence']
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_qnli_features(example_batch):
    #print(example_batch)
    inputs = list(zip(example_batch['question'], example_batch['sentence']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_wnli_features(example_batch):
    #print(example_batch)
    inputs = list(zip(example_batch['sentence1'], example_batch['sentence2']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_mnli_features(example_batch):
    #print(example_batch)
    inputs = list(zip(example_batch['premise'], example_batch['hypothesis']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_stsb_features(example_batch):
    inputs = list(zip(example_batch['sentence1'], example_batch['sentence2']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_rte_features(example_batch):
    inputs = list(zip(example_batch['sentence1'], example_batch['sentence2']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
    return features

def convert_to_commonsense_qa_features(example_batch):
    num_examples = len(example_batch["question"])
    num_choices = len(example_batch["choices"][0]["text"])
    features = {}
    for example_i in range(num_examples):
        choices_inputs = tokenizer.batch_encode_plus(
            list(zip(
                [example_batch["question"][example_i]] * num_choices,
                example_batch["choices"][example_i]["text"],
            )),
            max_length=max_length, pad_to_max_length=True,
        )
        for k, v in choices_inputs.items():
            if k not in features:
                features[k] = []
            features[k].append(v)
    labels2id = {char: i for i, char in enumerate("ABCDE")}
    # Dummy answers for test
    if example_batch["answerKey"][0]:
        features["labels"] = [labels2id[ans] for ans in example_batch["answerKey"]]
    else:
        features["labels"] = [0] * num_examples    
    return features

In [6]:
convert_func_dict = {
    "stsb": convert_to_stsb_features,
    "cola": convert_to_cola_features,
    "wnli": convert_to_wnli_features
    #"mnli": convert_to_mnli_features,
}

In [7]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased',do_lower_case=False)
columns_dict = {
    "stsb": ['input_ids', 'attention_mask', 'labels'],
    "cola": ['input_ids', 'attention_mask', 'labels'],
    "wnli": ['input_ids', 'attention_mask', 'labels'],
    #"mnli": ['input_ids', 'attention_mask', 'labels'],
    
}

features_dict = {}
for task_name, dataset in dataset_dict.items():
    features_dict[task_name] = {}
    for phase, phase_dataset in dataset.items():
        features_dict[task_name][phase] = phase_dataset.map(
            convert_func_dict[task_name],
            batched=True,
            load_from_cache_file=False,
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
        features_dict[task_name][phase].set_format(
            type="torch", 
            columns=columns_dict[task_name],
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))

  0%|          | 0/2 [00:00<?, ?it/s]

cola train 8551 8551
cola train 8551 8551


 50%|█████     | 1/2 [00:00<00:00,  5.28it/s]

cola validation 1043 1043
cola validation 1043 1043


  0%|          | 0/6 [00:00<?, ?it/s]

...

stsb validation 1500 1500
stsb validation 1500 1500


  0%|          | 0/1 [00:00<?, ?it/s]

stsb test 1379 1379
stsb test 1379 1379


100%|██████████| 1/1 [00:00<00:00, 10.54it/s]

wnli train 635 635
wnli train 635 635
wnli validation 71 71
wnli validation 71 71
wnli test 146 146
wnli test 146 146


In [8]:
class NLPDataCollator(DataCollator):
    """
    Extending the existing DataCollator to work with NLP dataset batches
    """
    def collate_batch(self, features: List[Union[InputDataClass, Dict]]) -> Dict[str, torch.Tensor]:
        first = features[0]
        if isinstance(first, dict):
          # NLP data sets current works presents features as lists of dictionary
          # (one per example), so we  will adapt the collate_batch logic for that
            if "labels" in first and first["labels"] is not None:
                if first["labels"].dtype == torch.int64:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
                else:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.float)
                batch = {"labels": labels}
            for k, v in first.items():
                if k != "labels" and v is not None and not isinstance(v, str):
                    batch[k] = torch.stack([f[k] for f in features])
            return batch
        else:
          # otherwise, revert to using the default collate_batch
          return DefaultDataCollator().collate_batch(features)


class StrIgnoreDevice(str):
    """
    This is a hack. The Trainer is going call .to(device) on every input
    value, but we need to pass in an additional `task_name` string.
    This prevents it from throwing an error
    """
    def to(self, device):
        return self


class DataLoaderWithTaskname:
    """
    Wrapper around a DataLoader to also yield a task name
    """
    def __init__(self, task_name, data_loader):
        self.task_name = task_name
        self.data_loader = data_loader

        self.batch_size = data_loader.batch_size
        self.dataset = data_loader.dataset

    def __len__(self):
        return len(self.data_loader)
    
    def __iter__(self):
        for batch in self.data_loader:
            batch["task_name"] = StrIgnoreDevice(self.task_name)
            yield batch


class MultitaskDataloader:
    """
    Data loader that combines and samples from multiple single-task
    data loaders.
    """
    def __init__(self, dataloader_dict):
        self.dataloader_dict = dataloader_dict
        self.num_batches_dict = {
            task_name: len(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        self.task_name_list = list(self.dataloader_dict)
        self.dataset = [None] * sum(
            len(dataloader.dataset) 
            for dataloader in self.dataloader_dict.values()
        )

    def __len__(self):
        return sum(self.num_batches_dict.values())

    def __iter__(self):
        """
        For each batch, sample a task, and yield a batch from the respective
        task Dataloader.

        We use size-proportional sampling, but you could easily modify this
        to sample from some-other distribution.
        """
        task_choice_list = []
        for i, task_name in enumerate(self.task_name_list):
            task_choice_list += [i] * self.num_batches_dict[task_name]
        task_choice_list = np.array(task_choice_list)
        np.random.shuffle(task_choice_list)
        dataloader_iter_dict = {
            task_name: iter(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        for task_choice in task_choice_list:
            task_name = self.task_name_list[task_choice]
            yield next(dataloader_iter_dict[task_name])    

class MultitaskTrainer(transformers.Trainer):

    def get_single_train_dataloader(self, task_name, train_dataset):
        """
        Create a single-task data loader that also yields task names
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        if is_tpu_available():
            train_sampler = get_tpu_sampler(train_dataset)
        else:
            train_sampler = (
                RandomSampler(train_dataset)
                if self.args.local_rank == -1
                else DistributedSampler(train_dataset)
            )

        data_loader = DataLoaderWithTaskname(
            task_name=task_name,
            data_loader=DataLoader(
              train_dataset,
              batch_size=self.args.train_batch_size,
              sampler=train_sampler,
              collate_fn=self.data_collator.collate_batch,
            ),
        )

        if is_tpu_available():
            data_loader = pl.ParallelLoader(
                data_loader, [self.args.device]
            ).per_device_loader(self.args.device)
        return data_loader

    def get_train_dataloader(self):
        """
        Returns a MultitaskDataloader, which is not actually a Dataloader
        but an iterable that returns a generator that samples from each 
        task Dataloader
        """
        return MultitaskDataloader({
            task_name: self.get_single_train_dataloader(task_name, task_dataset)
            for task_name, task_dataset in self.train_dataset.items()
        })

In [9]:
start=time()
print('Training started at ',start)
train_dataset = {
    task_name: dataset["train"] 
    for task_name, dataset in features_dict.items()
}
trainer = MultitaskTrainer(
    model=multitask_model,
    args=transformers.TrainingArguments(
        output_dir="./models/bert_multitask",
        overwrite_output_dir=True,
        learning_rate=5e-5,
        do_train=True,
        num_train_epochs=3,
        # Adjust batch size if this doesn't fit on the Colab GPU
        per_device_train_batch_size=8,  
        save_steps=3000,
        logging_steps=100,
        logging_dir='bert_logs'
    ),
    data_collator=NLPDataCollator(),
    train_dataset=train_dataset,
)
trainer.train()
print(time()-start)


Training started at  1608178710.4154248


{"loss": 1.534033979922533, "learning_rate": 4.9107780157030696e-05, "epoch": 0.05353319057815846, "step": 100}
{"loss": 1.138928828984499, "learning_rate": 4.821556031406139e-05, "epoch": 0.10706638115631692, "step": 200}
{"loss": 0.8458010891079902, "learning_rate": 4.732334047109208e-05, "epoch": 0.16059957173447537, "step": 300}
{"loss": 0.8967944318056107, "learning_rate": 4.643112062812277e-05, "epoch": 0.21413276231263384, "step": 400}
{"loss": 0.8474469895660878, "learning_rate": 4.553890078515346e-05, "epoch": 0.2676659528907923, "step": 500}
{"loss": 0.8665631264448166, "learning_rate": 4.4646680942184155e-05, "epoch": 0.32119914346895073, "step": 600}
{"loss": 0.7265155212581158, "learning_rate": 4.375446109921485e-05, "epoch": 0.3747323340471092, "step": 700}
{"loss": 0.7698795759677887, "learning_rate": 4.286224125624554e-05, "epoch": 0.4282655246252677, "step": 800}
{"loss": 0.7649577705562115, "learning_rate": 4.1970021413276235e-05, "epoch": 0.4817987152034261, "step": 

{"loss": 0.5431072661280631, "learning_rate": 3.3047822983583155e-05, "epoch": 1.0171306209850106, "step": 1900}
{"loss": 0.4937980341725051, "learning_rate": 3.215560314061385e-05, "epoch": 1.0706638115631693, "step": 2000}
{"loss": 0.4528332656994462, "learning_rate": 3.126338329764454e-05, "epoch": 1.1241970021413277, "step": 2100}
{"loss": 0.4278562973998487, "learning_rate": 3.0371163454675235e-05, "epoch": 1.177730192719486, "step": 2200}
{"loss": 0.4323692122101784, "learning_rate": 2.9478943611705928e-05, "epoch": 1.2312633832976445, "step": 2300}
{"loss": 0.4581507395673543, "learning_rate": 2.8586723768736618e-05, "epoch": 1.284796573875803, "step": 2400}
{"loss": 0.42963651043362916, "learning_rate": 2.769450392576731e-05, "epoch": 1.3383297644539613, "step": 2500}
{"loss": 0.45649848332628606, "learning_rate": 2.6802284082798e-05, "epoch": 1.39186295503212, "step": 2600}
{"loss": 0.4699536261707544, "learning_rate": 2.5910064239828698e-05, "epoch": 1.4453961456102784, "step

/home/ecbm4040/anaconda3/envs/mtxln-pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 0.40913829740136864, "learning_rate": 2.2341184867951464e-05, "epoch": 1.6595289079229123, "step": 3100}
{"loss": 0.4401936921104789, "learning_rate": 2.1448965024982158e-05, "epoch": 1.7130620985010707, "step": 3200}
{"loss": 0.4513427553512156, "learning_rate": 2.0556745182012848e-05, "epoch": 1.7665952890792291, "step": 3300}
{"loss": 0.4185695594362915, "learning_rate": 1.966452533904354e-05, "epoch": 1.8201284796573876, "step": 3400}
{"loss": 0.4556513046566397, "learning_rate": 1.8772305496074234e-05, "epoch": 1.873661670235546, "step": 3500}
{"loss": 0.4320819187629968, "learning_rate": 1.7880085653104924e-05, "epoch": 1.9271948608137044, "step": 3600}
{"loss": 0.38856534026563166, "learning_rate": 1.6987865810135617e-05, "epoch": 1.9807280513918628, "step": 3700}



{"loss": 0.3449959940928966, "learning_rate": 1.609564596716631e-05, "epoch": 2.0342612419700212, "step": 3800}
{"loss": 0.2619784353300929, "learning_rate": 1.5203426124197004e-05, "epoch": 2.08779443254818, "step": 3900}
{"loss": 0.26835629214998336, "learning_rate": 1.4311206281227696e-05, "epoch": 2.1413276231263385, "step": 4000}
{"loss": 0.24683412973303348, "learning_rate": 1.3418986438258387e-05, "epoch": 2.194860813704497, "step": 4100}
{"loss": 0.247724237262737, "learning_rate": 1.252676659528908e-05, "epoch": 2.2483940042826553, "step": 4200}
{"loss": 0.24813799444586038, "learning_rate": 1.1634546752319772e-05, "epoch": 2.3019271948608138, "step": 4300}
{"loss": 0.25699892740696667, "learning_rate": 1.0742326909350464e-05, "epoch": 2.355460385438972, "step": 4400}
{"loss": 0.26233643759973346, "learning_rate": 9.850107066381155e-06, "epoch": 2.4089935760171306, "step": 4500}
{"loss": 0.24313153212191538, "learning_rate": 8.95788722341185e-06, "epoch": 2.462526766595289, "s

In [ ]:
# from time import time
# start=time()
# print('Training started at ',start)
# train_dataset = {
#     task_name: dataset["train"] 
#     for task_name, dataset in features_dict.items()
# }
# trainer = MultitaskTrainer(
#     model=multitask_model,
#     args=transformers.TrainingArguments(
#         output_dir="./models/multitask_model",
#         overwrite_output_dir=True,
#         learning_rate=5e-5,
#         do_train=True,
#         num_train_epochs=3,
#         # Adjust batch size if this doesn't fit on the Colab GPU
#         per_device_train_batch_size=32,  
#         save_steps=3000,
#     ),
#     data_collator=NLPDataCollator(),
#     train_dataset=train_dataset,
# )
# trainer.train(optimizer='adamax')
# print(time()-start)


In [ ]:
print((time()-start) / 60)

In [10]:
preds_dict = {}
for task_name in ["cola", "stsb", "wnli"]:
    eval_dataloader = DataLoaderWithTaskname(
        task_name,
        trainer.get_eval_dataloader(eval_dataset=features_dict[task_name]["validation"])
    )
    print(eval_dataloader.data_loader.collate_fn)
    preds_dict[task_name] = trainer._prediction_loop(
        eval_dataloader, 
        description=f"Validation: {task_name}",
    )
    #print(task_name,preds_dict[task_name])

<bound method NLPDataCollator.collate_batch of <__main__.NLPDataCollator object at 0x7f50a6076310>>



<bound method NLPDataCollator.collate_batch of <__main__.NLPDataCollator object at 0x7f50a6076310>>



<bound method NLPDataCollator.collate_batch of <__main__.NLPDataCollator object at 0x7f50a6076310>>


In [ ]:
! pip install scipy sklearn

In [ ]:
# Evalute RTE
# nlp.load_metric('glue', name="cola").compute(
#     np.argmax(preds_dict["cola"].predictions, axis=1),
#     preds_dict["cola"].label_ids,
# )

cola_metric=nlp.load_metric('glue', name='cola')

In [ ]:
np.all(np.argmax(preds_dict["cola"].predictions, axis=1) == 1)

In [ ]:
cola_metric.compute()

In [ ]:
nlp.load_metric('glue', name="stsb").compute(
    preds_dict["stsb"].predictions.flatten(),
    preds_dict["stsb"].label_ids,
)

In [ ]:
preds_dict["cola"].label_ids

In [ ]:
# Evalute Commonsense QA
np.mean(
    np.argmax(preds_dict["wnli"].predictions, axis=1)
    == preds_dict["wnli"].label_ids
)

In [ ]:
from matplotlib import pyplot as plt

In [11]:
scores = {}

scores['cola'] = nlp.load_metric('glue', name='cola').compute(
    np.argmax(preds_dict['cola'].predictions, axis=1),
    preds_dict['cola'].label_ids
)
scores['stsb'] = nlp.load_metric('glue', name="stsb").compute(
    preds_dict["stsb"].predictions.flatten(),
    preds_dict["stsb"].label_ids,
)
scores['wnli'] = nlp.load_metric('glue', name='wnli').compute(
    np.argmax(preds_dict['wnli'].predictions, axis=1),
    preds_dict['wnli'].label_ids
)

In [12]:
scores

{'cola': {'matthews_correlation': 0.5208447598175431},
 'stsb': {'pearson': 0.8678433501501895, 'spearmanr': 0.8655824809199388},
 'wnli': {'accuracy': 0.4507042253521127}}

In [14]:
import pickle 

with open('bert_scores.pkl', 'wb') as fd :
    pickle.dump(scores, fd)